In [1]:
import pandas as pd
import re
from datetime import datetime
from IPython.display import FileLink


### Define each of these fields each time based on the column names in the raw data

In [2]:
# DEFINE THESE EACH TIME YOU RUN THE SCRIPT

# filepath for the template and raw files
template= '1Bulk_sample_template_V11.xlsx'
raw= 'PRB_LB_0325_raw.xlsx'
shipping_manifest= 'PRB_LB_0325_shipping_manifest.xlsx'
dataset= 'PRB_LB_0325'
output_path= f"{dataset}_formatted.xlsx"

# sheet name for the raw file
sheet_name_raw= 'Clinical Data'
sheet_name_shipping= 'Template'


# header row for the raw and template files
raw_header= 1
template_header= 1
shipping_manifest_header= 10

# columns in the raw file that contain biomarker data
biomarker_cols= ["Biomarker 1", "Biomarker 2", "Biomarker 3", "Biomarker 4", "Biomarker 5", "Biomarker 6"]

# column mappings from raw to template (key: template column, value: raw column - add more columns as needed). If blank, default value will be Not received
column_mapping= {
    "ExternalId": "Sample ID\nconsecutive",
    'TubeBarcode': 'Barcode Scan',
    "Stabilizer": "Collection Tube Type",
    "Single or Double Spun": "Single or Double Spun",
    "Hemolysis": "Hemolysis",
    "SpecimenType": "Sample   Type",
    "Date of Blood Draw/Cell Collection": "Date of blood collection [yyyy-mm-dd]",
    "Time of Draw": "Date of blood collection [yyyy-mm-dd]",
    "Gender": "Gender",
    "Height": "Body height [cm]",
    "Sample Timepoint": "Blood sample timepoint",
    "TNM": "TNM",
    "Stage": "Stage",
    "Morphology Code": "Morphology code",
    "Description of Morphology Code": "Description of morphology code",
    "ExPatientId": "Patient ID\nconsecutive",
    "ExSpecaminId": "Sample ID\nconsecutive",
}

# fixed values that aren't in the raw file (key: template column, value: fixed value)
fixed_values= {
    "ContainerType": "tube",
    "Organism": "Human",
    "Project": "PB-2881",
    "Data Transformer": "Nalika Palayoor",
    "Date of Transformation": "17-Jul-2025",
    "Condition": "cancer",
    "Diagnostic Condition": "breast cancer",
    "RNA-Sequencing Available": "No",
    "Source": "Indivumed",
    "Country": "Germany"
}

# define as True if menopausal status is to be extracted from biomarker data
extract_menopause_from_biomarker= True





### Cleaning functions

In [3]:
# function factory for basic mapping cleaning
def make_cleaner(mapping_dict):
    """
    Given a mapping dictionary, returns a cleaning function that maps raw values to the standard values
    Parameters:
        mapping_dict (dict): dictionary where keys are standard values and values are lists of raw value options
    Returns: 
        function: takes a single value and returns the cleaned value or pd.NA if not found
    """
    lookup= {}
    for standard_val, raw_options in mapping_dict.items():
        for raw_val in raw_options:
            lookup[str(raw_val).strip().lower()]= standard_val
    def cleaner(val):
        if pd.isna(val):
            return pd.NA
        val_str= str(val).strip().lower()
        return lookup.get(val_str, pd.NA)
    return cleaner

# function to extract menopausal status from biomarker columns (logic set so it only runs if specified)
def extract_menopause_status(row):
    """
    Given a row of data, extracts menopausal status from biomarker columns if present
    Parameters:
        row (pd.Series): a row of the dataframe that may contain menopause status
    Returns: 
        str or pd.NA: cleaned menopausal status or pd.NA if not found
    """

    for col in biomarker_cols:
        val= row.get(col)
        if pd.isna(val):
            continue
        val_str= str(val).strip().lower()

        if "menopause status" in val_str:
            parts= val_str.split("=")
            if len(parts)>1:
                raw_status= parts[1].strip().lower()

                for clean_val, options in menopause_mapping.items():
                    if raw_status in [opt.lower() for opt in options]:
                        return clean_val
                return "Unknown"
    return pd.NA

# function to interpret HER2 status from complex strings
def interpret_her2_status(val):
    """
    Given a raw HER2 status string, returns the cleaned HER2 status
    Parameters:
        val (str): raw HER2 status string
    Returns: 
        str or pd.NA: cleaned HER2 status ("positive", "negative", "HER2 2+ (FISH/ISH missing)") or pd.NA if not found
    """
    if pd.isna(val):
        return pd.NA

    val = str(val).lower()
    her2_score = None
    fish_result = None

    match_ihc = re.search(r"her2\s*=\s*([0-3]\+?)", val)
    if match_ihc:
        her2_score = match_ihc.group(1)

    if "fish" in val or "ish" in val:
        if "positive" in val:
            fish_result = "positive"
        elif "negative" in val:
            fish_result = "negative"

    if her2_score in ["0", "1+"]:
        return "negative"
    elif her2_score == "3+":
        return "positive"
    elif her2_score == "2+":
        if fish_result == "positive":
            return "positive"
        elif fish_result == "negative":
            return "negative"
        else:
            return "HER2 2+ (FISH/ISH missing)"

    return pd.NA

# function to extract and clean biomarker data from biomarker columns
def extract_pos_neg_biomarker(row, biomarker_cols, biomarker_lookup, pos_neg_mapping):
    """
    Extracts positive and negative biomarker results from a row of data.
    Parameters:
        row (pd.Series): a row of the dataframe containing biomarker information
        biomarker_cols (list): list of columns to check for biomarker data
        biomarker_lookup (dict): mapping of biomarker names to standardized names
        pos_neg_mapping (dict): mapping of raw results to positive/negative labels
    Returns:
        results: a dictionary of extracted biomarker results
    """
    results = {}

    for col in biomarker_cols:
        cell= row.get(col)
        if pd.isna(cell):
            continue
        val= str(cell).strip().lower()

        if "=" in val:
            biomarker_name, raw_result= [s.strip() for s in val.split("=", 1)]

            for template_col, raw_variants in biomarker_lookup.items():
                if biomarker_name.lower() in [v.lower() for v in raw_variants]:
                    if template_col== "HER2":
                        mapped = interpret_her2_status(val)
                    else:
                        mapped = None
                        for label, values in pos_neg_mapping.items():
                            for v in values:
                                if v.lower() in raw_result:
                                    mapped= label
                                    break
                            if mapped:
                                break
                        if not mapped:
                            mapped= raw_result
                        try:
                            num= int(raw_result)
                            if biomarker_name.lower()== 'er':
                                results['ER Value']= num
                            if biomarker_name.lower()== 'pr':
                                results['PR Value']= num
                        except:
                            pass

                    results[template_col]= mapped
    return results

# functions to put the date and time in the correct format
def clean_date(val):
    """
    Cleans and formats a date string.
    Parameters:
        val (str): raw date string
    Returns:
        str or pd.NA: cleaned date string in "YYYY-MM-DD" format or pd.NA if not found
    """
    if pd.isna(val):
        return pd.NA

    try:
        return pd.to_datetime(val).strftime("%Y-%m-%d")
    except:
        return pd.NA

def clean_time(val):
    """
    Cleans and formats a time string.
    Parameters:
        val (str): raw time string
    Returns:
        str or pd.NA: cleaned time string in "HH:MM:SS AM/PM" format or pd.NA if not found
    """
    if pd.isna(val):
        return pd.NA

    try:
        return pd.to_datetime(val).strftime("%I:%M:%S %p")
    except: 
        return pd.NA

# functions for columns that need to be calculated
def calculate_age(birth_val, collection_date):
    """
    Calculates age in years given birth date (or year) and collection date. If only year is given for birth date, assumes Jan 1 of that year.
    Parameters:
        birth_val (str): raw birth date string
        collection_date (str): raw collection date string
    Returns:
        int or pd.NA: age in years or pd.NA if not calculable
    """
    if pd.isna(birth_val) or pd.isna(collection_date):
        return pd.NA
    try:
        birth_str= str(birth_val).strip()
        if len(birth_str)== 4 and birth_str.isdigit():
            birth_date= pd.to_datetime(f"{birth_str}-01-01")
        else:
            birth_date= pd.to_datetime(birth_str, errors= "coerce")
        collection_dt= pd.to_datetime(collection_date, errors= "coerce")

        age= (collection_dt-birth_date).days // 365
        return age
    except:
        return pd.NA

def calculate_elapsed_days(row, start_col, end_col):
    """
    Calculates elapsed days between two date columns in a row.
    Parameters:
        row (pd.Series): a row of the dataframe containing date information
        start_col (str): name of the column with the start date
        end_col (str): name of the column with the end date
    Returns:
        int or pd.NA: number of elapsed days or pd.NA if not calculable
    """
    start_val= row.get(start_col)
    end_val= row.get(end_col)

    if pd.isna(start_val):
        return pd.NA
    if pd.isna(end_val):
        return pd.NA
    try:
        start_date= pd.to_datetime(start_val, errors= "coerce")
        end_date= pd.to_datetime(end_val, errors= "coerce")

        if pd.isna(start_date) or pd.isna(end_date):
            return pd.NA

        delta= (end_date-start_date).days
        return delta if delta >= 0 else pd.NA
    except:
        return pd.NA



### Synonym dictionaries for data cleaning (add to these as you find new synonyms in the raw data)

In [4]:
# If there are new synonyms found in the raw data, don't delete the old ones, just add to the lists below

menopause_mapping= {
    "premenopause": ["pre menopause"],
    "perimenopause": ["peri menopause"],
    "menopause": ["menopause"],
    "postmenopause": ["post menopause"]
}

biomarker_lookup = {
    "HER2": ["her2"],
    "ER": ["er", "estrogen receptor"],
    "PR": ["pr", "progesterone receptor"],
    "FISH": ["fish"],
    "PDL1": ["pdl1"],
    "ALK": ["alk"],
    "ROS": ["ros"],
    "EGFR": ["egfr"],
    "KRAS": ["kras"],
    "PIK3CA": ["pik3ca"],
    "ESR1": ["esr1"],
    "AR": ["ar"],
}

pos_neg_mapping = {
    "Positive": ["positive", "strong positive", "weak positive", "2+", "3+", "1", "2","3","4","5","6","7","8","9","10","11","12","12",">= 10", ">=10"],
    "Negative": ["negative", "0", "none", "not detected", "less than 10%", "less than 10", "1+"]
    
}

# create functions from function factory (add to dictionaries as you find new synonyms in the raw data)
clean_stabilizer= make_cleaner({
    "Streck": ["Streck Cell-Free DNA BCT"]
})

clean_gender= make_cleaner({
    "Male": ['m', 'male', 'M', 'Male'],
    "Female": ['f', 'female', 'F', 'Female']
})

clean_single_double= make_cleaner({
    "Single": ['single', 'Single','1'],
    "Double": ['double', 'Double','2']
})

clean_sample_timepoint= make_cleaner({
    "treatment-naïve": ["Initial-0"]
})

clean_stage= make_cleaner({
    "I": ["I", "IA", "IB"],
    "II": ["II", "IIA", "IIB"],
    "III": ["III", "IIIA", "IIIB"],
    "IV": ["IV", "IVA", "IVB"]
})

# transformation registry (key: template column, value: cleaning function)
# add a new row to the dictionary for each new function you create/use
transformations= {
    "Date of Blood Draw/Cell Collection": clean_date,
    "Time of Draw": clean_time,
    "Stabilizer": clean_stabilizer,
    "Gender": clean_gender,
    "Single or Double Spun": clean_single_double,
    "Sample Timepoint": clean_sample_timepoint,
    "Stage": clean_stage
}

### Function calling + formatting df

In [5]:
# load in the data

template= pd.read_excel(template, header= template_header)
raw= pd.read_excel(raw, sheet_name= sheet_name_raw, header= raw_header)
shipping_manifest= pd.read_excel(shipping_manifest, sheet_name= sheet_name_shipping, header= shipping_manifest_header)
final= pd.DataFrame(index= raw.index, columns= template.columns)

# merge the raw data with the shipping manifest to get tube barcodes
raw= pd.merge(raw, shipping_manifest, left_on= 'Patient ID\nconsecutive',right_on= 'Patient ID consecutive', how= "left")

# this occurs if menopause status is in biomarker columns
if extract_menopause_from_biomarker:
    final["Menopausal Status"]= raw.apply(extract_menopause_status, axis=1)

# adding biomarker columns to the column mapping
biomarker_data = raw.apply(
    lambda row: extract_pos_neg_biomarker(row, biomarker_cols, biomarker_lookup, pos_neg_mapping),
    axis=1
)

for idx, result in biomarker_data.items():
    for col, val in result.items():
        final.at[idx, col] = val

# Create column for duration between cancer diagnosis+ and blood draw
final["Duration between Cancer Diagnosis and Blood Draw (days)"] = raw.apply(
    lambda row: calculate_elapsed_days(row, "Date of cancer diagnosis [yyyy-mm-dd]", "Date of blood collection [yyyy-mm-dd]"), axis=1)

# create a column for duration between tnm staging and blood draw
final["Duration between TNM Staging and Blood Draw (days)"] = raw.apply(
    lambda row: calculate_elapsed_days(row, "Date of TNM staging  [yyyy-mm-dd]", "Date of blood collection [yyyy-mm-dd]"), axis=1)

# create a column for age at the time of blood collection
final["AgeAtCollection"]= raw.apply(
    lambda row: calculate_age(row["Year of birth"], row["Date of blood collection [yyyy-mm-dd]"]), axis=1)


# apply transformations and mappings

for col,val in fixed_values.items():
    if col in final.columns:
        final[col]= val

for template_col in column_mapping:
    raw_col= column_mapping[template_col]
    if raw_col in raw.columns:
        if template_col in transformations:
            final[template_col]= raw[raw_col].apply(transformations[template_col])
        else:
            final[template_col]= raw[raw_col]

# define columns that can be left blank if no data is present
required_columns = [
    "External Id","Received Date","Organism","Gender","Height","Weight","BMI","Sample Timepoint","Stage","Morphology Code","Description of Morphology Code",
    "ExPatientId","ER","PR","HER2","FISH","ALK","ROS","EGFR","KRAS","PIK3CA","ESR1","AR","RNA-sequencing available ",
    "Source (called Site on Sapio)","Country","Gender", "Race","SmokingHistory","Number of years smoked or smoking","Smoking Notes"
]


# if a column is blank and not optional, fill with Not received
for col in final.columns:
    if col in required_columns:
        final[col]= final[col].fillna("Not received")

# display the dataframe
pd.set_option("display.max_columns",None)
final.head(6)


,ExternalId,Received Date,ContainerType,Volume_uL,TubeBarcode,Concentration,ConcentrationUnits,Organism,Stabilizer,Single or Double Spun,Processing Method,Processing Time(hrs),Freeze Thaw Status,Hemolysis,Project,Matched FFPE Available,Date of Blood Draw/Cell Collection,Time of Draw,Block Size,Tissue Size,Tissue Weight (mg),% Tumor,% Necrosis,Surgery Type,Tumor Tissue Type,Data Transformer,Date of Transformation,Other Sample Notes,ExSpecimenId,Collection Site,SpecimenType,Condition,Diagnostic Condition,Histology,Height,Weight,BMI,Duration between Cancer Diagnosis and Blood Draw (days),Duration between Metastatic Diagnosis and Blood Draw (days),Sample Timepoint,Sample Timepoint Description,AgeAtCollection,Detailed Anatomical Location,Grade,Tumor Size,TNM,Duration between TNM Staging and Blood Draw (days),Stage,Stage Detailed,Morphology Code,Description of Morphology Code,Metastatic Sites,Vehicle Control,Media Conditions,Additional Supplements to Media,Protocols for Harvesting Cell Lines,Blood collection date (days from birth),Number of lines of metastatic therapy at time of blood draw,Number of lines of chemotherapy at time of blood draw,Number of lines of anti-HER2 therapy at time of blood draw,Number of lines of endocrine therapy at time of blood draw,Overall Survival(months),Treatment Data,Progression Free Survival(months),Gestational Age at Collection,Fetus Sex,Menopausal Status,Blood Type,HER2 IHC,HER2,HER2 FISH,FISH Notes,HER2 Change from Previos Sample,ER,ER Notes,ER Value,ER Status Change from Previous Sample,PR,PR Notes,PR Value,PR Status Change from Previous Sample,AR,AR Notes,AR Status Change From Previous Sample,ROS,ALK,EGFR,EGFR Allele Information,PDL1,KRAS,PIK3CA,ESR1,BRCA1,BRCA2,FOLR1,Ki67,TROP2,HER3,MET,Tissue Factor (TF),NECTIN4,CEACAM5,LIV1,MESOTHELIN,B7H3,B7H4,FRalpha,DLL3,CDH6,PGR,PSA,KLF5,FGF/FGFR,Biomarker Notes,RNA-Sequencing Available,ExPatientId,Source,Country,Gender,Race,MedicalHistory,FamilyHistory,AlcoholHistory,SmokingHistory,Number of years smoked or smoking,Smoking Notes,Donor Notes,FISH,ExSpecaminId
0,408_BD T0,Not received,tube,NaN,LV1004950744,NaN,NaN,Human,Streck,Double,NaN,NaN,NaN,No,PB-2881,NaN,2017-12-20,11:29:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nalika Palayoor,17-Jul-2025,NaN,NaN,NaN,plasma,cancer,breast cancer,NaN,160.0,Not received,Not received,5,NaN,treatment-naïve,NaN,57,NaN,NaN,NaN,cT4c cN1 cM1,5,IV,NaN,8500/3,"Infiltrating duct carcinoma, NOS (C50._)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,postmenopause,NaN,NaN,positive,NaN,NaN,NaN,Negative,NaN,NaN,NaN,Negative,NaN,NaN,NaN,Not received,NaN,NaN,Not received,Not received,Not received,NaN,NaN,Not received,Not received,Not received,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,PRB_408,Indivumed,Germany,Female,Not received,NaN,NaN,NaN,Not received,Not received,Not received,NaN,Not received,408_BD T0
1,409_BD T0,Not received,tube,NaN,LV2001303058,NaN,NaN,Human,Streck,Double,NaN,NaN,NaN,No,PB-2881,NaN,2019-09-17,11:04:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nalika Palayoor,17-Jul-2025,NaN,NaN,NaN,plasma,cancer,breast cancer,NaN,160.0,Not received,Not received,33,NaN,treatment-naïve,NaN,74,NaN,NaN,NaN,rT0 pN0 pM1 GX,33,IV,NaN,8500/3,"Infiltrating duct carcinoma, NOS (C50._)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,postmenopause,NaN,NaN,positive,NaN,NaN,NaN,Negative,NaN,0,NaN,Negative,NaN,0,NaN,Not received,NaN,NaN,Not received,Not received,Not received,NaN,NaN,Not received,Not received,Not received,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,PRB_409,Indivumed,Germany,Female,Not received,NaN,NaN,NaN,Not received,Not received,Not received,NaN,Positive,409_BD T0
2,410_BD T0,Not received,tube,NaN,LV2000478378,NaN,NaN,Human,Streck,Double,NaN,NaN,NaN,No,PB-2881,NaN,2019-02-12,11:34:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nalika Palayoor,17-Jul-2025,NaN,NaN,NaN,plasma,cancer,breast cancer,NaN,175.0,Not received,Not received,28,NaN,

### Run this to download the dataframe as a excel workbook

In [6]:
final.to_excel(output_path, index=False)
FileLink(output_path)


/Users/nalikapalayoor/Documents/raw to formatted workflow/automation/PRB_LB_0325_formatted.xlsx

In [7]:
shipping_manifest

,#,Patient ID \npre Data,Label \npreData,Patient ID consecutive,Label consecutive,LV-Number,Entity,Sample Type,Quantity,Quantity Unit,Box,Position,Barcode Scan,Matches Barcode,Sample Damage?,Hemolysis,Label Readable Issue?,Label Information Issue?,"Location (freezer 2, shelf x, rack x, position x)"
0,1,Pat_002,002_BD T0-1,PRB_408,408_BD T0,LV1004950744,breast,plasma,1.0,ml,Box 1,A1,LV1004950744,MATCH,No,No,No,No,"Freezer 2, shelf 2, rack 4, box 9, position 39"
1,2,Pat_005,005_BD T0-1,PRB_409,409_BD T0,LV2001303058,breast,plasma,1.0,ml,Box 1,A2,LV2001303058,MATCH,No,No,No,No,"Freezer 2, shelf 2, rack 4, box 9, position 40"
2,3,Pat_006,006_BD T0-1,PRB_410,410_BD T0,LV2000478378,breast,plasma,1.0,ml,Box 1,A3,LV2000478378,MATCH,No,No,No,No,"Freezer 2, shelf 2, rack 4, box 9, position 41"
3,4,Pat_007,007_BD T0-1,PRB_411,411_BD T0,LV2000478505,breast,plasma,1.0,ml,Box 1,A4,LV2000478505,MATCH,No,No,No,No,"Freezer 2, shelf 2, rack 4, box 9, position 42"
4,5,Pat_008,008_BD T0-1,PRB_412,412_BD T0,LV2008151600,breast,plasma,1.0,ml,Box 1,A5,LV2008151600,MATCH,No,No,No,No,"Freezer 2, shelf 2, rack 4, box 9, position 43"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,58,Pat_075,075_BD T0-1,PRB_465,465_BD T0,LV2000805029,breast,plasma,1.0,ml,Box 1,E10,LV2000805029,MATCH,No,Light Hemolysis,No,No,"Freezer 2, shelf 2, rack 4, box 10, position 15"
58,59,Pat_076,076_BD T0-1,PRB_466,466_BD T0,LV2008114446,breast,plasma,1.0,ml,Box 1,E11,LV2008114446,MATCH,No,No,No,No,"Freezer 2, shelf 2, rack 4, box 10, position 16"
59,60,Pat_077,077_BD T0-1,PRB_467,467_BD T0,LV2001930605,breast,plasma,1.0,ml,Box 1,E12,LV2001930605,MATCH,No,No,No,No,"Freezer 2, shelf 2, rack 4, box 10, position 17"
60,61,Pat_078,078_BD T0-1,PRB_468,468_BD T0,LV1004677351,breast,plasma,1.0,ml,Box 1,F1,LV1004677351,MATCH,No,No,No,No,"Freezer 2, shelf 2, rack 4, box 10, position 18"


In [8]:
print(raw.columns.tolist())
print(shipping_manifest.columns.tolist())


['Patient ID', 'Sample ID', 'Patient ID\nconsecutive', 'Sample ID\nconsecutive', 'Blood sample timepoint', 'Date of blood collection [yyyy-mm-dd]', 'Amount of aliquots', 'Sample or data', 'Gender', 'Year of birth', 'Body height [cm]', 'Minimum weight [kg]', 'Maximum weight [kg] ', 'Date of cancer diagnosis [yyyy-mm-dd]', 'ICD code', 'Description of ICD code', 'Organ of primary tumor', 'NSCLC or SCLC', 'Date of metastatic disease [yyyy-mm-dd]', 'Metastatic sites at index (ICD-O-3)', 'Date of TNM staging  [yyyy-mm-dd]', 'TNM', 'Stage', 'Dignity', 'Morphology code', 'Description of morphology code', 'Biomarker 1', 'Biomarker 2', 'Biomarker 3', 'Biomarker 4', 'Biomarker 5', 'Biomarker 6', 'Surgery Date [yyyy-mm-dd]', 'Surgery Doc', 'Treatment classification', 'Therapeutic agent #1', 'Dosage therapeutic agent #1', 'Dosage unit therapeutic agent #1', 'Therapeutic agent #2', 'Dosage therapeutic agent #2', 'Dosage unit therapeutic agent #2', 'Therapeutic agent #3', 'Dosage therapeutic agent #3